## Things to try to increase LB score
1. Change way in which buffer region is constucted (Cross, Square, Ball)
1. Change submission csv method to transpose rows and cols

Within first 200 000, we have
* PERCENT zeros
* PERCENT ones
* ie, PERPERCENT rate of ones

....We should set threshold accordingly.

Estimated time for getting 25m rows: ESTIMATE

(https://zindi.africa/competitions/geoai-amazon-basin-secret-runway-detection-challenge/discussions/22422)